In [2]:
import os
import random
import numpy as np
import joblib
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet import MobileNet, preprocess_input
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from xgboost import XGBClassifier
from skimage.feature import hog
import cv2
from PIL import Image

In [3]:
def load_image(img_path, target_size=(128, 128)):
    # Load the image with PIL to handle different formats
    img = Image.open(img_path).convert("RGB")
    img = img.resize(target_size)
    
    # Convert to array and preprocess for MobileNet
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)
    return img

In [4]:
def load_dataset_from_multiple_paths(dataset_paths, target_size=(128, 128), sample_size=3000):
    images = []
    labels = []
    
    for dataset_path in dataset_paths:
        for label in ['real', 'ai']:
            folder_path = os.path.join(dataset_path, label)
            
            # Get a list of all images in the folder and sample 5000 randomly
            img_files = os.listdir(folder_path)
            sampled_files = random.sample(img_files, min(sample_size, len(img_files)))
            
            for img_name in sampled_files:
                img_path = os.path.join(folder_path, img_name)
                img = load_image(img_path, target_size=target_size)
                images.append(img)
                labels.append(0 if label == 'real' else 1)
                
    return np.array(images), np.array(labels)

# Example usage
dataset_paths = [r'E:\AI Image Detection\Ai Image Dec\Dataset', r"E:\AI Image Detection\program\datasets\train"]
images, labels = load_dataset_from_multiple_paths(dataset_paths)


c:\Users\theka\AppData\Local\Programs\Python\Python311\Lib\site-packages\PIL\Image.py:975: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


In [5]:
# Global feature extraction using MobileNet
mobilenet_model = MobileNet(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

def extract_global_features(images):
    global_features = []
    for img in images:
        features = mobilenet_model.predict(img)
        features = features.flatten()
        global_features.append(features)
    return np.array(global_features)

global_features = extract_global_features(images)

# Local feature extraction using HOG
def extract_hog_features(images):
    hog_features = []
    for img in images:
        gray_img = cv2.cvtColor(np.squeeze(img).astype('uint8'), cv2.COLOR_RGB2GRAY)
        features = hog(gray_img, pixels_per_cell=(8, 8), cells_per_block=(2, 2), feature_vector=True)
        hog_features.append(features)
    return np.array(hog_features)

local_features = extract_hog_features(images)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 754ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━

In [6]:
features = np.hstack((global_features, local_features))

In [7]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

In [8]:
# Define individual models
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
svm_model = SVC(probability=True, kernel='linear', random_state=42)
gb_model = GradientBoostingClassifier(n_estimators=100, random_state=42)
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42, max_depth=3, n_estimators=50)
lr_model = LogisticRegression(max_iter=1000, random_state=42)

# Create a Voting Classifier with the ensemble of models
ensemble_model = VotingClassifier(estimators=[
    ('rf', rf_model),
    ('svm', svm_model),
    ('gb', gb_model),
    ('xgb', xgb_model),
    ('lr', lr_model)
], voting='soft')

# Train the ensemble model
ensemble_model.fit(X_train, y_train)

c:\Users\theka\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:158: UserWarning: [08:03:40] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0ed59c031377d09b8-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


VotingClassifier(estimators=[('rf', RandomForestClassifier(random_state=42)),
                             ('svm',
                              SVC(kernel='linear', probability=True,
                                  random_state=42)),
                             ('gb',
                              GradientBoostingClassifier(random_state=42)),
                             ('xgb',
                              XGBClassifier(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None, device=None,
                                            early_stopping_ro...
                                            learning_rate=None, max_bin=None,
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=3,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=50, n_jobs=None,
                                            num_parallel_tree=None,
                                            random_state=42, ...)),
                             ('lr',
                              LogisticRegression(max_iter=1000,
                                                 random_state=42))],
                 voting='soft')

In [4]:
# Predict and evaluate
y_pred = ensemble_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy * 100:.2f}%")
print("Classification Report:\n", report)

NameError: name 'ensemble_model' is not defined

In [3]:
# Save the model
model_filename = 'ai_image_detection_ensemble_model.joblib'
joblib.dump(ensemble_model, model_filename)
print(f"Ensemble model saved as {model_filename}")

NameError: name 'joblib' is not defined

In [4]:
model_filename = 'ai_image_detection_ensemble_model.joblib'
def load_model(filename):
    return joblib.load(filename)

def predict_image(img_path, model, target_size=(128, 128)):
    img = load_image(img_path, target_size=target_size)
    global_feat = extract_global_features([img])
    local_feat = extract_hog_features([img])
    combined_feat = np.hstack((global_feat, local_feat))
    prediction = model.predict(combined_feat)
    return 'AI-generated' if prediction[0] == 1 else 'Real'

# Load the model and test on a new image
loaded_model = load_model(model_filename)
new_image_path = r"C:\Users\theka\OneDrive\Desktop\tree.jpg"
print("Prediction:", predict_image(new_image_path, loaded_model))

NameError: name 'extract_global_features' is not defined